In [57]:
import numpy as np
import tensorflow as tf
import keras
from keras import Model, activations, layers, losses, optimizers, callbacks, regularizers
from keras.layers import Dense, Convolution1D, Dropout, BatchNormalization, concatenate, TimeDistributed, Layer, Input
import matplotlib.pyplot as plt
from ss_functions import *
from ss_pred_classes import *

In [24]:
x_data, y_data = get_data('list.txt')
x_train, y_train = x_data[101:], y_data[101:]
fff, y_data = get_data('list.txt', encode_y=False)
x_test, y_test = x_data[:101], y_data[:101]

In [58]:
x_data, y_data = get_data1('list.txt',padding=False)
x_train2, y_train2 = x_data[101:], y_data[101:]
fff, y_data = get_data1('list.txt', encode_y=False,padding=False)
x_test2, y_test2 = x_data[:101], y_data[:101]

NameError: name 'get_data1' is not defined

In [ ]:
x_data1, y_data1 = get_data2('list.txt')
x_train1, x_train2 = np.array(x_data1[0][101:]), np.array(x_data1[1][101:])
x_test1, x_test2 = np.array(x_data1[0][:101]), np.array(x_data1[1][:101])

In [39]:
def get_model0():
    inputs = Input((None, 41))
    inputs = layers.Masking()(inputs)
    X = DeepInception_block()(inputs)
    X = DeepInception_block()(X)
    # X = DeepInception_block()(X)

    X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
    X = TimeDistributed(Dense(512, activation='relu'))(X)
    X = Dropout(0.4)(X)
    X = TimeDistributed(Dense(256, activation='relu'))(X)
    X = Dropout(0.4)(X)

    Y = TimeDistributed(Dense(3, activation='softmax'))(X)
    model = Model(inputs=inputs, outputs=Y)
    return model

In [42]:
model = get_model0()

opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=['accuracy', 'mae', truncated_accuracy])
model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, None, 41)]        0         
                                                                 
 deep_inception_block_29 (De  (None, None, 900)        1278692   
 epInception_block)                                              
                                                                 
 deep_inception_block_30 (De  (None, None, 900)        2062100   
 epInception_block)                                              
                                                                 
 conv1d_1630 (Conv1D)        (None, None, 100)         990100    
                                                                 
 time_distributed_28 (TimeDi  (None, None, 512)        51712     
 stributed)                                                      
                                                          

In [44]:
stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_model0.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

lr_fun = callbacks.LearningRateScheduler(
    learn_decay
)

with tf.device('/GPU:1'):
    history = model.fit(x_data, y_data,
                        epochs=100,
                        batch_size=24,
                        validation_split=0.1,
                        callbacks=[stop_monitor_loss, checkpoint])
    
# 3rd block - negligeable improvement

Epoch 1/100


ValueError: in user code:

    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\vinicius\anaconda3\envs\tf\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
   

In [13]:
test_model = keras.models.load_model('ss_pred_model0.keras', 
                                     custom_objects={
                                         'inception_conv': inception_conv,
                                         'InceptionNet_paper': InceptionNet_paper,
                                         'DeepInception_block': DeepInception_block,
                                         'truncated_accuracy': truncated_accuracy
                                     })

In [14]:
ss_map = {'C': 0, 'H': 1, 'E': 2}
from_aa = {0: 'C', 1: 'H', 2: 'E'}
predictions_hot = test_model.predict(x_test)
predictions = []
for prediction in predictions_hot:
    dssp = ''
    for i in prediction:
        dssp += from_aa[np.argmax(i)]
    predictions.append(dssp)

total = 0
TP = 0
for prediction, truth in zip(predictions, y_test):
    for i, ss in enumerate(truth):
        total +=1
        if ss==prediction[i]:
            TP+=1

accuracy = TP/total
print(accuracy)

4/4 [==============================] - 5s 251ms/step
0.47987097574592147


In [ ]:
block1 = DeepInception_block()
block2 = DeepInception_block()
block3 = DeepInception_block()

inputs = layers.Input((800,41))
X = block1(inputs)
X = block2(X)
X1 = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X1 = TimeDistributed(Dense(256, activation='relu'))(X1)
X1 = Dropout(0.5)(X1)

Y2 = TimeDistributed(Dense(3, activation='softmax'))(X1)
X = block3(X)

# input1 = layers.Input((800,21))
# input2 = layers.Input((800,20))
# X1 = block1(input1)
# X2 = block2(input2)

# X = layers.concatenate([X1,X2])
# X = DeepInception_block()(X)

X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X = TimeDistributed(Dense(256, activation='relu'))(X)
X = Dropout(0.5)(X)

Y1 = TimeDistributed(Dense(3, activation='softmax'))(X)

model = Model(inputs=inputs, outputs=[Y1,Y2])

opt = optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=[truncated_accuracy])

stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_modeltest9.keras',
    monitor='val_truncated_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)


input:
1. one hot encoded sequence
2. PSSM

Model:
1D convolutional neural network

output:
multiclass classification - dense layer with relu activaiton - 3?

validation metric - accuray + model specific measures

soruces:
https://www.csbj.org/article/S2001-0370(22)00506-2/fulltext
